In [ ]:
import numpy as np
import pandas as pd
from IPython.display import display
from scipy.signal import resample
from scipy import signal
from tf.transformations import euler_from_quaternion
from sklearn.metrics import r2_score

# Load methods to extract data from rosbags
from unpack_rosbag import unpack_bag, synchronize_topics, correct_time_diff
from imu_ramp_offline import ImuRampDetect

# Standard plotly imports
import plotly.express as px
import plotly.io as pio

pio.templates.default = "plotly_dark"

In [ ]:
def get_data(bag_name, imu_topic):
    bag_path = BAG_DIRECTORY + bag_name
    # IMU data (angular velocity and linear acceleration)
    imu, t_imu = unpack_bag(bag_path, IMU_TOPIC, "imu")
    quat, _ = unpack_bag(bag_path, IMU_TOPIC, "quat")
    # Odom data (wheel speeds and handwheel angle)
    odom, t_odom = unpack_bag(bag_path, ODOM_TOPIC, "car_odom")
    # Odom data from hdl slam
    odom_hdl = unpack_bag(bag_path, ODOMHDL_TOPIC, "hdl_odom")

    # Ignore odom if none has been recorded
    if len(odom) == 0:
        f = 400
        # Align all signals in time (closest start and end time for all signals)
        data, time = correct_time_diff((imu, t_imu), odom_hdl)
        # Unpack values
        imu, odom_hdl = data
        odom = np.zeros((len(imu), 5))
        if imu_topic == "/imu/data":
            f = 100
    else:
        # Align all signals in time (closest start and end time for all signals)
        data, time = correct_time_diff((imu, t_imu), (odom, t_odom), odom_hdl)
        # Unpack values
        imu, odom, odom_hdl = data
        t_imu, t_odom = time[:2]
        # Downsamples higher freq imu signal to match odom data
        imu, odom = synchronize_topics(imu, t_imu, odom, t_odom)
        f = 100

    # Split linear acceleration and angular velocity
    ang_vel = imu[:, 0]
    lin_acc = imu[:, 1]

    t = np.linspace(0, odom.shape[0] / f, odom.shape[0])

    return lin_acc, ang_vel, odom, odom_hdl, t, f


def run_algo(lin_acc, ang_vel, odom, f):
    ird = ImuRampDetect(f)
    tf, gyr_bias = ird.align_imu(lin_acc, ang_vel, odom)
    lin_accs = []
    ang_vels = []
    angles = []
    for i, _ in enumerate(lin_acc):
        la, av, angs, _ = ird.spin(lin_acc[i], ang_vel[i], odom[i], tf, gyr_bias)
        lin_accs.append(la)
        ang_vels.append(av)
        # angles.append(angs)
    return lin_accs, ang_vels


def pitch_from_lidar(odom_hdl, size):
    euler_angles = []
    for i, ori in enumerate(odom_hdl):
        q = [ori.orientation.x, ori.orientation.y, ori.orientation.z, ori.orientation.w]
        euler_angle = euler_from_quaternion(q)
        euler_angles.append(euler_angle)
    euler_angles = -np.rad2deg(np.asarray(euler_angles))
    pitch_angle = upsample(euler_angles[:, 1], size)
    return pitch_angle


def upsample(sig, size):
    # Get the flipped signal
    flip = np.flip(sig)
    # Mirror signal at the beginning at start and end
    sig_extended = np.hstack((flip, sig, flip))
    # Resample signal (*3 due to adding two flipped parts)
    sig_ext_up = resample(sig_extended, size * 3)
    # Get original signal again
    sig_up = np.split(sig_ext_up, 3)[1]
    return sig_up


# Evaluation
def RMSE(y, y_pred):
    return np.sqrt(((y_pred - y) ** 2)).mean()


def calc_score(angles_pred, angle_ref, ramp_type):
    scores = []
    # Convert from radians to degree
    angles_pred = np.asarray(np.rad2deg(angles_pred))
    methods = ["acc", "gyr", "acc_odom", "compl", "compl_grav"]
    df_angles = pd.DataFrame(angles_pred, columns=methods)
    for i in range(df_angles.shape[1]):
        ang = df_angles.iloc[:, i]
        rmse = RMSE(angle_ref, ang)
        r2 = r2_score(angle_ref, ang)
        scores.append((ramp_type, methods[i], rmse, r2))
    return scores


def run_evaluation(bag_info):
    """Get scores from all bags"""
    scores = []
    for _, v in enumerate(bag_info):
        # Unpack dictionary
        bag_name = v["bag_name"]
        ramp_type = v["ramp_type"]
        print("\nLoading bag {}".format(bag_name))
        lin_acc, ang_vel, odom, odom_hdl, t, f = get_data(bag_name, IMU_TOPIC)
        print("Running algorithm")
        lin_acc_car, ang_vel_car = run_algo(lin_acc, ang_vel, odom, f)
        # Add angle from lidar
        lidar_ang = pitch_from_lidar(odom_hdl, len(lin_acc_car))
        score = calc_score(angles, lidar_ang, ramp_type)
        scores.append(score)


# Rosbags path
BAG_DIRECTORY = "/home/user/rosbags/final/slam/"
# ROS topics
ODOM_TOPIC = "/eGolf/sensors/odometry"
ODOMHDL_TOPIC = "/odom"
IMU_TOPIC = "/zed2i/zed_node/imu/data"
# IMU_TOPIC = "/imu/data"

# List of dictionaries with info about recording
BAG_INFO = [
    {
        "bag_name": "d_d2r2s_lidar_wo_odom_hdl.bag",
        "ramp_type": "d_full_long",
    },
    {
        "bag_name": "d_d2r2s_odom_hdl.bag",
        "ramp_type": "d_full_long",
    },
    {
        "bag_name": "d_e2q_hdl.bag",
        "ramp_type": "d_full_short",
    },
    {
        "bag_name": "straight_wo_ramps_odom_hdl.bag",
        "ramp_type": "straight",
    },
    {
        "bag_name": "u_c2s_half_odom_hdl.bag",
        "ramp_type": "u_half_short",
    },
    {
        "bag_name": "u_c2s_half_odom_stereo_hdl.bag",
        "ramp_type": "u_half_short",
    },
    {
        "bag_name": "u_c2s_hdl.bag",
        "ramp_type": "u_full_short",
    },
    {
        "bag_name": "u_c2s_stop_hdl.bag",
        "ramp_type": "u_full_short",
    },
    {
        "bag_name": "u_d2e_hdl.bag",
        "ramp_type": "u_full_short",
    },
    {
        "bag_name": "u_s2c_half_odom_hdl.bag",
        "ramp_type": "u_half_short",
    },
    {
        "bag_name": "u_s2c2d_hdl.bag",
        "ramp_type": "u_full_long",
    },
]

# Run evaluation
scores = []
linear_acc = []
angular_vel = []
lidar_angs = []
odoms = []
g_mags = []
fs = []
for _, v in enumerate(BAG_INFO):
    # Unpack dictionary
    bag_name = v["bag_name"]
    ramp_type = v["ramp_type"]
    print("\nLoading bag {}".format(bag_name))
    lin_acc, ang_vel, odom, odom_hdl, t, f = get_data(bag_name, IMU_TOPIC)
    print("Running algorithm")
    lin_acc_car, ang_vel_car = run_algo(lin_acc, ang_vel, odom, f)
    odoms.append(odom)
    linear_acc.append(lin_acc_car)
    angular_vel.append(ang_vel)
    g_mags.append(np.linalg.norm(np.mean(lin_acc_car[:f], axis=0)))
    # Add angle from lidar
    lidar_ang = pitch_from_lidar(odom_hdl, len(lin_acc_car))
    lidar_angs.append(lidar_ang)
    fs.append(f)
    if _== 1:
        break

In [ ]:
# Convert all measurements in car frame to np array
la = [np.asarray(x) for x in linear_acc]
av = [np.asarray(x) for x in angular_vel]

In [ ]:
def moving_average(sig, win, f):
    # Convert window size from s to idx
    win = int(win * f)
    b = np.ones(win) / win
    a = np.ones(1)
    return signal.filtfilt(b, a, sig)

fig = go.Figure()
fig.add_trace(go.Scatter(y = y_pred))
fig.add_trace(go.Scatter(y = moving_average(y_pred, 0.02, 100)))

In [ ]:
def vel_from_odom(odom, track_width = 1.52):
    """Car velocity from wheel speeds"""
    alpha = (odom[:, 2] - odom[:, 3]) / track_width
    yaw = alpha * 1 / f
    # 3.6 to convert from km/h to m/s
    vel_x_car = ((odom[:, 2] + odom[:, 3]) / 2) * np.cos(yaw) / 3.6
    return vel_x_car

def acc_from_vel(sig):
    return np.diff(sig, prepend=0) / (1.0 / f)

def moving_average(sig, win):
    # Convert window size from s to idx
    win = int(win * f)
    b = np.ones(win) / win
    a = np.ones(1)
    return signal.lfilter(b, a, sig)

def pitch_from_grav(acc_x_imu, odom, win):
    # Calculate car velocity
    vel_x_car = vel_from_odom(odom)
    # Calculate acceleration of filtered velocity
    acc_x_car_filt = acc_from_vel(moving_average(vel_x_car, win))
    acc_x_imu_filt = moving_average(acc_x_imu, win)
    acc_x_fuse = acc_x_imu_filt - acc_x_car_filt
    return np.rad2deg(np.arcsin(acc_x_fuse / g_mag))

def pitch_from_acc(acc_x_imu, g_mag):
    return np.rad2deg(np.arcsin(acc_x_imu / g_mag))

In [ ]:
rmses = []
r2s = []
wins = np.arange(0.02, 2, 0.1)
for win in wins:
    rmse = []
    r2 = []
    for i, v in enumerate(la):
        y_pred = moving_average(pitch_from_acc(la[i][:-fs[i],0], g_mags[i]), win, fs[i])
        y_true = lidar_angs[i][:-fs[i]]
        rmse.append(RMSE(y_true, y_pred))
        r2.append(r2_score(y_true, y_pred))
    rmses.append(np.mean(rmse))
    r2s.append(np.mean(r2))

print("Best rmse: {:.3f} with delay={}".format(np.min(rmses), wins[np.argmin(rmses)]))
print("Best r2: {:.3f} with delay={}".format(np.max(r2s), wins[np.argmax(r2s)]))

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(y = y_true[:-fs[i]]))
fig.add_trace(go.Scatter(y = moving_average(y_pred[:-fs[i]], 1.72, 100)))

In [ ]:
import plotly.graph_objects as go
import matplotlib.pyplot as plt

In [ ]:
def shift_signal(y_orig, y_delayed, delay, f):
    return y_delayed[delay*f:]

In [ ]:
y_del = np.copy(y_true)

In [ ]:
np.pad(y_del, (1*f, 0))

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(y = y_true))
fig.add_trace(go.Scatter(y = shift_signal(y_del, 1, f)))

In [ ]:
np.correlate(y_true, moving_average(pitch_from_acc(la[i][:,0], g_mags[i]), 0.41))

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(y = y_true))
fig.add_trace(go.Scatter(y = moving_average(pitch_from_acc(la[i][:,0], g_mags[i]), 0.5)))

In [ ]:
px.line(y_pred)

In [ ]:
rmses = []
r2s = []
k_vals = np.arange(1, 0.25, -0.05)
for k in k_vals:
    rmse = []
    r2 = []
    for i, v in enumerate(la):
        y_pred = complementary_filter(la[i][:,0], av[i][:,1], fs[i], k)
        y_true = lidar_angs[i]
        rmse.append(RMSE(y_true, y_pred) + 10*RMSE(y_true[-fs[i]:], y_pred[-fs[i]:]))
        r2.append(r2_score(y_true, y_pred))
    rmses.append(np.mean(rmse))
    r2s.append(np.mean(r2))

print("Best rmse: {:.3f} with K={}".format(np.min(rmses), k_vals[np.argmin(rmses)]))
print("Best r2: {:.3f} with K={}".format(np.max(r2s), k_vals[np.argmax(r2s)]))

In [ ]:
def complementary_filter(acc_x_imu, vel_y_imu, f, K):
    # Delay
    # K = tau / (tau + 1.0/f)
    # initial estimations
    g_mag = 9.81
    angle_k = 0
    angle_est = []
    for i, v in enumerate(acc_x_imu):
        # Calculate pitch angle using linear acceleration data
        theta_a = np.arcsin(acc_x_imu[i] / g_mag)
        angle_k = K * (angle_k - vel_y_imu[i] / f) + (1 - K) * theta_a
        # Ignore acc readings if they are unreliable
        angle_est.append(angle_k)
    return np.rad2deg(angle_est)